In [1]:
import datetime
from sklearn.base import clone
from sklearn.model_selection import KFold,train_test_split
import numpy as np
import pandas as pd
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import accuracy_score
from colorama import Fore, Style
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, SplineTransformer, OneHotEncoder
from sklearn.linear_model import Ridge, LinearRegression
import matplotlib.pyplot as plt
import copy
from catboost import CatBoostClassifier,CatBoostRegressor,CatBoostClassifier
from lightgbm import LGBMRegressor,LGBMClassifier
from xgboost import XGBRFRegressor,XGBClassifier
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.preprocessing import LabelEncoder

In [2]:
train = pd.read_csv('dataset/train.csv', index_col='id')
test = pd.read_csv('dataset/test.csv', index_col='id')
#train = pd.read_csv('/kaggle/input/playground-series-s4e5/train.csv', index_col='id')
#test = pd.read_csv('/kaggle/input/playground-series-s4e5/test.csv', index_col='id')

initial_features = list(train.columns)[:-1]
label_transform = LabelEncoder()

# y = train["Target"].copy().astype(float)
y = label_transform.fit_transform(train["Target"])
train = train.drop(columns=["Target"])

# sorted_features = [f"sort_{i}" for i in np.arange(len(initial_features))]
# for df in [train, test]:
#     df[sorted_features] = np.sort(df[initial_features], axis=1)

X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2, shuffle=True, random_state=420)

In [3]:
def get_summ_info(df):
    print(f'data shape: {df.shape}')
    summ = pd.DataFrame(df.dtypes, columns=['data type'])
    summ['#missing'] = df.isnull().sum().values
    summ['%missing'] = df.isnull().sum().values / len(df) * 100
    summ['#unique'] = df.nunique().values
    desc = pd.DataFrame(df.describe(include='all').transpose())
    summ['min'] = desc['min'].values
    summ['max'] = desc['max'].values
#     summ['first value'] = df.loc[0].values
#     summ['second value'] = df.loc[1].values
#     summ['third value'] = df.loc[2].values
    return summ.style.background_gradient(cmap='Blues')

In [4]:
get_summ_info(X_train)

data shape: (61214, 36)


,data type,#missing,%missing,#unique,min,max
Marital status,int64,0,0.000000,6,1.000000,6.000000
Application mode,int64,0,0.000000,22,1.000000,53.000000
Application order,int64,0,0.000000,7,0.000000,6.000000
Course,int64,0,0.000000,19,33.000000,9991.000000
Daytime/evening attendance,int64,0,0.000000,2,0.000000,1.000000
Previous qualification,int64,0,0.000000,20,1.000000,43.000000
Previous qualification (grade),float64,0,0.000000,105,95.000000,190.000000
Nacionality,int64,0,0.000000,18,1.000000,109.000000
Mother's qualification,int64,0,0.000000,34,1.000000,44.000000
Father's qualification,int64,0,0.000000,39,1.000000,44.000000


In [5]:
model=CatBoostClassifier(task_type='GPU',verbose=False)
model.fit(X_train,y_train)
sigma1 = model.predict(X_test)
sigma1 = sigma1.reshape(-1)
print('r2_score',accuracy_score(y_test, sigma1))

r2_score 0.8346184004181914


In [7]:
model=LGBMClassifier()
model.fit(X_train,y_train)
sigma2 = model.predict(X_test)
print('r2_score',accuracy_score(y_test, sigma2))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002842 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1372
[LightGBM] [Info] Number of data points in the train set: 61214, number of used features: 36
[LightGBM] [Info] Start training from score -1.105283
[LightGBM] [Info] Start training from score -1.627388
[LightGBM] [Info] Start training from score -0.749843
r2_score 0.8340303188708834


In [8]:
model = XGBClassifier()
model.fit(X_train,y_train)
sigma3 = model.predict(X_test)
print('r2_score',accuracy_score(y_test, sigma3))

r2_score 0.8331155253528489


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
model = CatBoostClassifier(task_type='GPU', verbose=False)
param_grid = {
    'depth': [8,9],
    # 'learning_rate' : [0.01, 0.05, 0.1, 0.005],
    'iterations'    : [70, 100, 130]
}

# 创建GridSearchCV对象
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid.fit(train, y)
best_model = grid.best_estimator_
print(f"最优参数: {grid.best_params_}")
print(f"最优评分: {grid.best_score_}")

In [ ]:
model=CatBoostClassifier()
model.fit(train,y)
sigma2 = model.predict(X_test)
print('r2_score',accuracy_score(y_test, sigma2))

In [ ]:
sigma = model.predict(test)
sigma = label_transform.inverse_transform(sigma)
sub = pd.Series(sigma, index=test.index, name='Target')
filename = 'submission.csv'
sub.to_csv(filename)